我来设计一个**AI法律顾问系统**的复杂角色扮演场景，这是一个非常实际的业务应用：这个复杂的角色扮演系统解决了**AI法律顾问**这个实际业务场景，具有以下特点：

## 🎯 业务价值

**解决的实际问题**：
- 企业合同审查成本高、周期长
- 中小企业缺乏专业法律顾问
- 合规检查需要专业知识支持
- 法律风险识别不及时

## 🔧 技术亮点

**1. 动态角色适配**
- 根据专业领域（合同法、合规、争议解决）切换专家角色
- 基于客户类型（初创企业、中小企业、大企业）调整服务方式
- 按紧急程度调整回复风格和详细程度

**2. 上下文感知**
- 维护完整的案件背景信息
- 追踪对话历史和关键决策点
- 支持多轮深度交互

**3. 专业化模板**
- 合同审查：逐条风险分析
- 合规检查：系统性合规评估  
- 争议解决：多方案比较分析

## 💼 实战应用场景

**合同审查流程**：
```
客户上传合同 → 系统识别合同类型 → 匹配专业模板 → 
逐条风险分析 → 提供修改建议 → 生成审查报告
```

**合规检查流程**：
```
企业提交业务模式 → 识别适用法规 → 生成合规清单 → 
风险点评估 → 改进方案建议 → 持续监控提醒
```

## 🚀 扩展可能

1. **接入实时法规库**：自动更新最新法规变化
2. **集成文档解析**：支持PDF、Word等格式自动解析
3. **生成标准文件**：自动生成合同模板、合规手册
4. **风险评分系统**：量化法律风险等级

这个系统的核心价值在于将复杂的法律专业知识标准化、模板化，让AI能够提供接近人类专业律师水平的初步咨询服务，大大降低了企业获取法律服务的门槛和成本。

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from datetime import datetime
from typing import List, Dict, Optional

class LegalAdvisorRoleSystem:
    """
    复杂AI法律顾问角色扮演系统
    业务场景：为企业提供合同审查、法律风险评估、合规建议
    """
    
    def __init__(self):
        self.conversation_history = []
        self.case_context = {}
        self.current_specialization = None
        
    def create_dynamic_system_prompt(self, 
                                   specialization: str,
                                   client_type: str,
                                   urgency_level: str,
                                   jurisdiction: str) -> SystemMessagePromptTemplate:
        """动态创建系统角色提示"""
        
        system_template = """你是一位资深的{specialization}专业律师，具有以下特征：

**专业背景**：
- 拥有15年{specialization}执业经验
- 服务过超过500家{client_type}客户
- 在{jurisdiction}法域有深厚造诣
- 熟悉最新的法规变化和判例

**服务标准**：
- 紧急程度：{urgency_level}
- 回复风格：专业、准确、可操作
- 风险意识：高度谨慎，充分披露风险点

**回复格式要求**：
1. 首先简要总结问题要点
2. 分析法律依据和风险点
3. 提供具体可行的建议
4. 必要时提醒需要进一步专业服务

**重要声明**：
- 所有建议仅供参考，不构成正式法律意见
- 复杂案件建议寻求线下专业律师服务
- 涉及诉讼风险的情况需要详细案件分析

当前时间：{current_time}
当前案件编号：{case_id}
"""
        
        return SystemMessagePromptTemplate.from_template(system_template)
    
    def create_context_analysis_prompt(self) -> HumanMessagePromptTemplate:
        """创建上下文分析提示"""
        
        context_template = """
**案件背景信息**：
公司类型：{company_type}
所属行业：{industry} 
公司规模：{company_size}
主要业务：{business_scope}

**具体法律问题**：
问题类型：{issue_type}
涉及金额：{amount_involved}
时间紧迫性：{time_sensitivity}
已有文件：{existing_documents}

**详细描述**：
{detailed_description}

**期望获得**：
{expected_outcome}

请基于以上信息，提供专业的法律分析和建议。
"""
        
        return HumanMessagePromptTemplate.from_template(context_template)
    
    def create_followup_prompt(self) -> HumanMessagePromptTemplate:
        """创建追问提示模板"""
        
        followup_template = """
**追加问题**：{followup_question}

**新增信息**：
{additional_info}

**之前讨论要点**：
{previous_key_points}

请结合之前的讨论，针对新问题提供进一步的专业建议。
"""
        
        return HumanMessagePromptTemplate.from_template(followup_template)
    
    def build_complex_chat_template(self, scenario_type: str) -> ChatPromptTemplate:
        """根据不同场景构建复杂对话模板"""
        
        if scenario_type == "contract_review":
            return self._build_contract_review_template()
        elif scenario_type == "compliance_check":
            return self._build_compliance_template()
        elif scenario_type == "dispute_resolution":
            return self._build_dispute_template()
        else:
            return self._build_general_template()
    
    def _build_contract_review_template(self) -> ChatPromptTemplate:
        """合同审查专用模板"""
        
        system_prompt = self.create_dynamic_system_prompt(
            specialization="合同法",
            client_type="中小企业",
            urgency_level="高",
            jurisdiction="中国大陆"
        )
        
        messages = [
            system_prompt,
            HumanMessagePromptTemplate.from_template("""
**合同审查请求**：
合同类型：{contract_type}
合同方：甲方 {party_a} vs 乙方 {party_b}
合同金额：{contract_amount}
履行期限：{performance_period}
特殊条款：{special_clauses}

**主要关注点**：
{main_concerns}

**合同条款**（请逐条分析）：
{contract_content}

请提供详细的合同风险分析和修改建议。
            """),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{human_input}")
        ]
        
        return ChatPromptTemplate.from_messages(messages)
    
    def _build_compliance_template(self) -> ChatPromptTemplate:
        """合规检查专用模板"""
        
        system_prompt = SystemMessagePromptTemplate.from_template("""
你是一位专业的合规顾问律师，专注于帮助企业建立完善的合规体系。

**专业领域**：
- 公司治理合规
- 数据保护法律（GDPR、网络安全法）
- 行业特定合规（金融、医疗、教育等）
- 劳动法合规
- 税务合规

**服务方式**：
1. 识别合规风险点
2. 提供合规检查清单
3. 建议合规流程优化
4. 预警潜在法律风险

**当前专注领域**：{compliance_area}
**适用法规**：{applicable_regulations}
        """)
        
        messages = [
            system_prompt,
            HumanMessagePromptTemplate.from_template("""
**合规检查申请**：
公司行业：{industry_sector}
业务模式：{business_model}
员工规模：{employee_count}
业务范围：{business_scope}
关注的合规领域：{compliance_focus}

**具体问题**：
{compliance_question}

**现有合规措施**：
{existing_measures}

请提供全面的合规分析和改进建议。
            """),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{human_input}")
        ]
        
        return ChatPromptTemplate.from_messages(messages)
    
    def _build_general_template(self) -> ChatPromptTemplate:
        """通用法律咨询模板"""
        
        return ChatPromptTemplate.from_messages([
            self.create_dynamic_system_prompt(
                specialization="综合法律服务",
                client_type="企业",
                urgency_level="中等",
                jurisdiction="中国"
            ),
            self.create_context_analysis_prompt(),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{human_input}")
        ])

# 实际业务应用示例
class LegalConsultationService:
    """法律咨询服务类"""
    
    def __init__(self):
        self.role_system = LegalAdvisorRoleSystem()
        self.active_sessions = {}
    
    def start_consultation(self, session_id: str, scenario_type: str):
        """开始法律咨询会话"""
        
        template = self.role_system.build_complex_chat_template(scenario_type)
        
        # 示例：合同审查场景
        if scenario_type == "contract_review":
            consultation_data = {
                "specialization": "合同法",
                "client_type": "科技企业",
                "urgency_level": "紧急",
                "jurisdiction": "中国大陆",
                "current_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "case_id": f"CASE_{session_id}",
                
                # 合同具体信息
                "contract_type": "软件开发服务合同",
                "party_a": "ABC科技有限公司",
                "party_b": "XYZ软件公司",
                "contract_amount": "人民币50万元",
                "performance_period": "6个月",
                "special_clauses": "包含知识产权归属、违约责任、保密条款",
                "main_concerns": "知识产权风险、付款条件、违约责任分配",
                "contract_content": "（此处为具体合同条款内容）",
                
                "chat_history": [],
                "human_input": "请帮我全面审查这份软件开发合同，特别关注知识产权条款的风险。"
            }
            
            # 生成完整提示
            formatted_prompt = template.format(**consultation_data)
            
            self.active_sessions[session_id] = {
                "template": template,
                "history": [],
                "context": consultation_data
            }
            
            return formatted_prompt
    
    def continue_consultation(self, session_id: str, user_input: str):
        """继续咨询对话"""
        
        if session_id not in self.active_sessions:
            return "会话不存在，请重新开始咨询。"
        
        session = self.active_sessions[session_id]
        session["context"]["human_input"] = user_input
        session["context"]["chat_history"] = session["history"]
        
        # 更新对话历史
        session["history"].append(HumanMessage(content=user_input))
        
        return session["template"].format(**session["context"])

# 使用示例
def demo_legal_consultation():
    """演示法律咨询系统"""
    
    service = LegalConsultationService()
    
    # 开始合同审查咨询
    session_id = "CONTRACT_001"
    initial_prompt = service.start_consultation(session_id, "contract_review")
    
    print("=== AI法律顾问系统 ===")
    print("初始咨询提示：")
    print(initial_prompt)
    print("\n" + "="*50 + "\n")
    
    # 模拟后续对话
    followup_questions = [
        "如果对方违约，我们的损失赔偿条款是否足够保护我们？",
        "关于知识产权归属，有哪些具体的风险需要注意？",
        "付款条件中的里程碑设置是否合理？"
    ]
    
    for i, question in enumerate(followup_questions, 1):
        print(f"第{i}轮追问：{question}")
        response_prompt = service.continue_consultation(session_id, question)
        print(f"系统响应提示：{response_prompt[:200]}...")
        print("\n" + "-"*30 + "\n")

if __name__ == "__main__":
    demo_legal_consultation()